# Convert PDF to CSV, with pdfplumber

### High light:
1. for very organized PDF files, plumber might be a smart, fast way to convert files

In [ ]:
import glob
import os
import pdfplumber
import pandas as pd
directory = r"C:\Users\46798566/Box\Py_codes\TexasCourt\Book\Attorney_list"
den_files = (
            glob.glob(os.path.join(directory, "AttorneyPopulationDensity*.pdf")) +
            glob.glob(os.path.join(directory, "PopulationDensityReport*.pdf"))
            )
#pdf_text = open("extracted_text.txt").read()

### do one file first

In [ ]:
# set up what kinda table will be extracted:
table_settings = {
    "vertical_strategy": "text",
    "horizontal_strategy": "text",
    # "intersection_tolerance": 5,
    # "snap_tolerance": 3,
    # "join_tolerance": 6,
    # "edge_min_length": 3,
}

# to detect the county names only...
table_settings2 = {
    "vertical_strategy": "lines",
    "horizontal_strategy": "text",
    # "intersection_tolerance": 5,
    # "snap_tolerance": 3,
    # "join_tolerance": 6,
    # "edge_min_length": 3,
}

In [ ]:
def select_rows_per_page(df):   # find out which col has county names
    county_counts = (
        df.astype(str)
          .apply(lambda col: col.str.contains(r"\bCount", case=True, na=False).sum())
          .sort_values(ascending=False)
    )
    county_col = county_counts.idxmax()
    df_reduced = df[
        df[county_col]
          .astype(str)
          .str.contains(r"\bCount", case=True, na=False)  # 'count, county, etc
    ]
    #print('there are' , len(df_reduced), "rows in the returned table. There were", len(df), 'rows...')
    if len(df_reduced) > 0 :
        print('Counties include:', df_reduced[county_col].iloc[0], 'and other', len(df_reduced)-1, 'counties')
    else:
        print('No counties in this "table". There were', len(df)-1, 'lines before.')
    return(df_reduced)

#table = tables[0]    #

target_file = den_files[3]
pdf =  pdfplumber.open(target_file)

len(all_tables_on_pdf)
table_df = pd.DataFrame(all_tables_on_pdf[10]) 

# extract all the tables....
all_tables_on_pdf = []
all_tables_on_pdf_para = []
for page in pdf.pages:
    tables = page.extract_tables(table_settings=table_settings) # no result, return [] 
    tables_para = page.extract_tables(table_settings=table_settings2) # no result, return []   # the two strategies should have the same number of rows.. table para contains the name of counties. 
    print("Now processing page ", page.page_number, ".", sep = "")
    all_tables_on_pdf.extend(tables)
    all_tables_on_pdf_para.extend(tables_para)

# then we have a huge list of tables...
all_tables_on_pdf_reduced = []
num_tables = len(all_tables_on_pdf)
for x in range(num_tables):
    table_df = pd.DataFrame(all_tables_on_pdf[x])
    #table_para_df = pd.DataFrame(all_tables_on_pdf_para[x])

    print(len(table_df), len(table_para_df))
    #if (len(table_df) == len(table_para_df)):
    #    table_df = pd.concat([table_df, table_para_df], axis=1)
    #else:
    #    pass
    table_df_reduced = select_rows_per_page(table_df)
    all_tables_on_pdf_reduced.append(table_df_reduced)
all_tables_as_df = pd.concat(all_tables_on_pdf_reduced, ignore_index = True)
clean_df = clean_table(all_tables_as_df)

clean_df.to_csv("table_test.csv")


pd.all_tables_as_df
print(all_tables_as_df)
df = pd.DataFrame(table)




df[3].str.contains(r'county', case = False)

df[1]
print(table)




with pdfplumber.open(target_file) as pdf:
    for p in range(start_page - 1, end_page):
        page = pdf.pages[p]
        tables = page.extract_tables(table_settings=table_settings) or []

### use ChatGPT:
i want for each row, when there were empty col, move the rest col to the left. then combine the col where there is name 'County' with the previous one, if the first letter is not . or Capitalized

In [ ]:
###### the following table is entirely defined by Chatgpt. Let me see if it works:
import pandas as pd
import numpy as np
import re

def _is_empty(x):
    if x is None:
        return True
    if isinstance(x, float) and np.isnan(x):
        return True
    return str(x).strip() == ""

def _shift_left_row(row_vals):
    vals = [v for v in row_vals if not _is_empty(v)]
    return vals + [None] * (len(row_vals) - len(vals))

def _needs_merge_county(cell):
    s = str(cell).strip()
    if "County" not in s:
        return False
    if not s:
        return False
    first = s[0]
    # merge when first letter is NOT '.' and NOT uppercase
    return (first != ".") and (not first.isupper())

def clean_table(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()

    # 1) shift left per row
    out = out.apply(lambda r: pd.Series(_shift_left_row(r.tolist()), index=out.columns), axis=1)

    # 2) merge "* County" fragments into previous cell (when fragment starts lowercase etc.)
    def merge_row(row):
        vals = row.tolist()
        for i in range(1, len(vals)):
            if _is_empty(vals[i]) or _is_empty(vals[i-1]):
                continue
            if _needs_merge_county(vals[i]):
                prev = str(vals[i-1]).rstrip()
                cur  = str(vals[i]).lstrip()
                # join without adding a space (works for ".Har" + "din County")
                vals[i-1] = re.sub(r"\s+", " ", (prev + cur)).strip()
                vals[i] = None
        # shift left again after merges
        return pd.Series(_shift_left_row(vals), index=row.index)

    out = out.apply(merge_row, axis=1)

    return out

In [ ]:
#### a more comprehensive version: 
# Row-wise left shift (remove empty cells by shifting remaining values left).
# Merge “County” fragments into the previous cell when the “County” cell starts with neither . nor uppercase (e.g., "an County").
# Merge stray dot cells: if a cell is exactly "." and the cell to its right starts with a capital letter, merge them (so "." + "Callah" → ".Callah"), then shift left again.

import pandas as pd
import numpy as np
import re

def _is_empty(x):
    if x is None:
        return True
    if isinstance(x, float) and np.isnan(x):
        return True
    return str(x).strip() == ""

def _shift_left(vals, width):
    keep = [v for v in vals if not _is_empty(v)]
    return keep + [None] * (width - len(keep))

def _starts_with_capital(x):
    s = str(x).strip()
    return len(s) > 0 and s[0].isupper()

def clean_table(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    ncol = out.shape[1]

    def clean_one_row(row):
        vals = row.tolist()

        # (1) shift left to remove empties
        vals = _shift_left(vals, ncol)

        # (2) merge "County" fragments into previous cell if first char is not '.' and not uppercase
        for i in range(1, ncol):
            if _is_empty(vals[i]) or _is_empty(vals[i-1]):
                continue
            s = str(vals[i]).strip()
            if "County" in s:
                first = s[0] if s else ""
                if (first != ".") and (not first.isupper()):
                    prev = str(vals[i-1]).rstrip()
                    cur  = str(vals[i]).lstrip()
                    vals[i-1] = re.sub(r"\s+", " ", (prev + cur)).strip()
                    vals[i] = None

        vals = _shift_left(vals, ncol)

        # (3) merge stray '.' with right cell if right starts with capital
        for i in range(0, ncol - 1):
            if str(vals[i]).strip() == "." and not _is_empty(vals[i+1]) and _starts_with_capital(vals[i+1]):
                vals[i] = "." + str(vals[i+1]).lstrip()
                vals[i+1] = None

        vals = _shift_left(vals, ncol)
        return pd.Series(vals, index=row.index)

    out = out.apply(clean_one_row, axis=1)
    return out

## next

In [5]:
import pdfplumber
import pandas as pd

pdf_path = r"C:\Users\46798566\Downloads\AttorneyPopulationDensity2014-15.pdf"
out_csv  = r"C:\Users\46798566\Downloads\AttorneyPopulationDensity2014-15_pages14-20_combined.csv"
out_notable = r"C:\Users\46798566\Downloads\AttorneyPopulationDensity2014-15_pages14-20_no_table_pages.txt"

start_page = 14
end_page   = 20  # inclusive

table_settings = {
    "vertical_strategy": "lines",
    "horizontal_strategy": "lines",
    "intersection_tolerance": 5,
    "snap_tolerance": 3,
    "join_tolerance": 3,
    "edge_min_length": 3,
}

def clean_df_from_table(table):
    df = pd.DataFrame(table)
    df = df.dropna(how="all").dropna(axis=1, how="all")
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

    # Use first row as header if it looks header-ish
    if len(df) >= 2:
        header = df.iloc[0].tolist()
        if sum(bool(h) for h in header) >= sum(bool(x) for x in df.iloc[1].tolist()):
            df.columns = header
            df = df.iloc[1:].reset_index(drop=True)

    return df

all_parts = []
no_table_pages = []
canonical_cols = None

with pdfplumber.open(pdf_path) as pdf:
    for p in range(start_page - 1, end_page):
        page = pdf.pages[p]
        tables = page.extract_tables(table_settings=table_settings) or []

        if len(tables) == 0:
            no_table_pages.append(p + 1)  # human page number
            continue

        for t in tables:
            df = clean_df_from_table(t)
            if df.empty:
                continue

999
defaultdict(<function <lambda> at 0x000002394E92ECA0>, {'x': 999})


In [6]:
import sys
sys.executable

'C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\python.exe'

In [ ]:
# read the table, and convert it too a united format (first make sure no missing values...)
# combine each tab to a big data frame, and then convert it to a wide format , make sure the names on the county stay the same


In [ ]:
import pandas as pd

excel_path = r"C:\Users\46798566\Downloads\Texas_counties.xlsx"   # <-- change this

# read all sheets
sheets = pd.read_excel(excel_path, sheet_name=None)

dfs = []

for year, df in sheets.items():
    y = df['year'].iloc[0]
    
    df = df.copy()
    dfs.append(df)

big_long = pd.concat(dfs, ignore_index=True)
big_long = big_long.iloc[:, :7]
big_long['year']  = pd.to_numeric(big_long.iloc[:, 0]).astype("Int64")


county_col = "county name"
value_cols = [c for c in big_long.columns if c not in [county_col, "year"]]

wide = big_long.pivot(
    index= county_col,
    columns= "year",
    values=value_cols
    #aggfunc="first"
)

# flatten column index
wide.columns = [f"{col}__{year}" for col, year in wide.columns]
wide = wide.reset_index()
big_long.to_csv("wide.csv")
wide.to_csv("wide.csv")
            

In [ ]:
# assume county column is literally named "county name"
county_col = "county name"
year_col = "year"
big_long[year_col] = pd.to_numeric(big_long[year_col], errors="coerce").astype("Int64")

value_cols = [c for c in big_long.columns if c not in [county_col, "year"]]


wide = big_long.pivot_table(
    index=county_col,
    columns=year_col,
    values=value_cols,
    aggfunc="first" 
)

# flatten column index
wide.columns = [f"{col}__{year}" for col, year in wide.columns]
wide = wide.reset_index()

wide